# Sentiment Classification of Product Reviews Using RNNs

## Read and Explore the dataset

Explore the dataset

Data Manipulations - Data Processing

Cleaning

Tokenize the reviews

Convert this into tensor. Array

Padding for small reviews

Build a model and train the model

Predict the sentiments


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [ ]:
%%writefile get_data.sh
if [ ! -f amazon_product_reviews_mod.csv ]; then
  wget -O amazon_product_reviews_mod.csv https://www.dropbox.com/scl/fi/imdwnlb4b617hfrepik8c/amazon_product_reviews_mod.csv?rlkey=3oafw8jsifa74v8hsvsuhcf77&dl=0
fi

Writing get_data.sh


In [ ]:
!bash get_data.sh

--2024-04-18 08:44:03--  https://www.dropbox.com/scl/fi/imdwnlb4b617hfrepik8c/amazon_product_reviews_mod.csv?rlkey=3oafw8jsifa74v8hsvsuhcf77
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucea131d5b02031700f9fe205fa6.dl.dropboxusercontent.com/cd/0/inline/CROOCimW2LRWEFCX4BISaPIwb8TEv8fPzispzHGf-892hGz-QR1WzyvMVwDbFQARzr2u5YzB68SYecfHs1o-vBv89hpFukwzI0quMPKNdhkWtHr-5KDmI-PzR4Q_hFQMlk0kX5EDs0P97sb2eSufsfNR/file# [following]
--2024-04-18 08:44:04--  https://ucea131d5b02031700f9fe205fa6.dl.dropboxusercontent.com/cd/0/inline/CROOCimW2LRWEFCX4BISaPIwb8TEv8fPzispzHGf-892hGz-QR1WzyvMVwDbFQARzr2u5YzB68SYecfHs1o-vBv89hpFukwzI0quMPKNdhkWtHr-5KDmI-PzR4Q_hFQMlk0kX5EDs0P97sb2eSufsfNR/file
Resolving ucea131d5b02031700f9fe205fa6.dl.dropboxusercontent.com (ucea131d5b02031700f9fe205fa6.dl.dropboxusercontent.com)... 16

In [ ]:
!ls

amazon_product_reviews_mod.csv	get_data.sh  sample_data


In [ ]:
!cat amazon_product_reviews_mod.csv

Streaming output truncated to the last 5000 lines.
5,"These batteries are an awesome price, and they work just the same as purchasing Energizer or Duracell. I see no reason to ever purchase those expensive brands again. I order the bulk sizes from Amazon and they keep me stocked up for quite a long time!",Work just as well as name brands!,ByJessica L Houze
4,"Cant see any difference from the more expensive Duracell batteries. Great price. Use them for flashlights clocks, and remote controls.",Work just as well as the more expensive batteries.,ByBJG
5,Work just fine,Work just fine,ByAmazon Customer
5,Wonderful batteries that seem to be well-worth the price we paid!,Work just fine!,ByNorth Carolina Consumers Council
5,"Work just like any other high end batteries I have purchased in the past but at a great price. Will buy more of these in the future, for sure!",Work just like any other high end batteries I have purchased in ...,ByLorena
5,"Work just as well as the name brand ones with the

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



We already downloaded the dataset from [here](https://www.kaggle.com/datasets/datafiniti/consumer-reviews-of-amazon-products?select=1429_1.csv) and saved it as a CSV file which we'll read now using Pandas.

In [ ]:
#path = 'HousePricePrediction.csv.csv'
df = pd.read_csv('amazon_product_reviews_mod.csv')

In [ ]:
df.shape

(28332, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviews.rating    28332 non-null  int64 
 1   reviews.text      28332 non-null  object
 2   reviews.title     28332 non-null  object
 3   reviews.username  28327 non-null  object
dtypes: int64(1), object(3)
memory usage: 885.5+ KB


In [ ]:
df.iloc[28000]

reviews.rating                                                      5
reviews.text        My mom had an old one, she was so excited to g...
reviews.title                                                 Perfect
reviews.username                                               Nagrom
Name: 28000, dtype: object

In [ ]:
print(df.iloc[100]["reviews.rating"])
print(df.iloc[100]["reviews.text"])

5
As a teacher, I need tons of batteries, but I refused to spend excessive amounts on them, so I figured this was the best option! They are long-lasting, worth the money, cute (if that even matters, lol)! You won't find a deal like this in stores!I highly recommend!


In [ ]:
print(df.iloc[27162]["reviews.rating"])
print(df.iloc[27162]["reviews.text"])

5
Had to chose between Fire 10 and Fire 8. For the money, you can't go wrong with the Fire 8. Screen is good for " older eyes".


## Data Manipulations

In [ ]:
reviews_df = df[["reviews.text", "reviews.rating",]]
reviews_df.columns = ["review", "rating"]

In [ ]:
reviews_df.tail(10)

,review,rating
28322,this product is really awesome for kids to pla...,4
28323,The 8 In Fire tablet does everything my nexus ...,4
28324,"Got this for my mom, so she could check facebo...",4
28325,"its fast, it has good lighting. its got the 16...",5
28326,"Where do I begin...good clarity, I love the si...",5
28327,I got 2 of these for my 8 yr old twins. My 11 ...,5
28328,I bought this for my niece for a Christmas gif...,4
28329,"Very nice for light internet browsing, keeping...",5
28330,This Tablet does absolutely everything I want!...,5
28331,"At ninety dollars, the expectionations are low...",4


In [ ]:
reviews_df.isnull().sum()

review    0
rating    0
dtype: int64

In [ ]:
reviews_df.dropna(inplace=True)

In [ ]:
def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "positive"
    elif rating == 3:
        return "neutral"
    elif (rating == 2) or (rating == 1):
        return "negative"

In [ ]:
reviews_df["sentiment"] = reviews_df["rating"].apply(sentiments)

In [ ]:
reviews_df.sample(10, random_state = 86, ignore_index=True)

,review,rating,sentiment
0,perfect for plane rides. the product operates ...,5,positive
1,This suited my exact needs which was a mobile ...,5,positive
2,Seem to last a long time,4,positive
3,I never owned a Kindle before. This was easy t...,5,positive
4,good batteries for the price. i will order again,5,positive
5,I realized after purchasing that the reason yo...,1,negative
6,"I do a lot of international travelling, so I h...",5,positive
7,Bought one of these two weeks before on Amazon...,5,positive
8,I reccomend this tabley yo every one. I purcha...,5,positive
9,My granddaughter loved her tablet and the colo...,5,positive


In [ ]:
sentiments = reviews_df["sentiment"].values
reviews = reviews_df["review"].values
print(sentiments[0:10])
print("\n")
print(reviews[0:10])

['neutral' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'neutral' 'positive']


['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.'
 'Bulk is always the less expensive way to go for products like these'
 'Well they are not Duracell but for the price i am happy.'
 'Seem to work as well as name brand batteries at a much better price'
 'These batteries are very long lasting the price is great.'
 "Bought a lot of batteries for Christmas and the AmazonBasics Cell have been good. I haven't noticed a difference between the brand name batteries and the Amazon Basic brand. Just a lot easier to purchase and have arrive at the house and have on hand. Will buy again."
 'ive not had any problame with these batteries have ordered them in the past been very pleased.'
 'Well if you are looking for cheap non-rechargeable batteries that last quite a while then these are perfect. No

## Convert text categories into one-hot encoded vector

We are importing keras utils here, this will be needed to convert the labels to one-hot encoded vectors. We'll use the the function "to_categorical" from this library to do this. To know more about this function, please check out this [link](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical).

1. Tensorflow
2. PyTorch
3. Keras

%pip install tensorflow

In [ ]:
import tensorflow.keras.utils as ku


Keras to-categorical function takes numerical arguments, that's why we need to convert the text labels to numerical values first. To achieve this, we'll define this funnction.

In [ ]:
def encode_sentiments(sentiment):
    if sentiment == "negative":
        return 0
    elif sentiment == "neutral":
        return 1
    elif sentiment == "positive":
        return 2

In [ ]:
label_encoding = {0: 'negative', 1: 'neutral', 2: 'positve'}

In [ ]:
sentiments_encoded = [encode_sentiments(sentiment) for sentiment in sentiments]

In [ ]:
print(sentiments[0:10])
print(sentiments_encoded[0:10])

['neutral' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'neutral' 'positive']
[1, 2, 2, 2, 2, 2, 2, 2, 1, 2]


In [ ]:
labels = ku.to_categorical(sentiments_encoded, num_classes = 3)

In [ ]:
print(labels[0:20])
sentiments[0:20]

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


array(['neutral', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'neutral', 'positive',
       'positive', 'neutral', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'positive', 'neutral'],
      dtype=object)

## Cleaning of the review texts

In [ ]:
test_str = "I am really really impressed by this product."
print(test_str)
old_pattern = "really"
new_pattern = "very"
new_str = re.sub(old_pattern, new_pattern, test_str)
print(new_str)

I am really really impressed by this product.
I am very very impressed by this product.


In [ ]:
test_str = "Python 3.8"
print(test_str)
old_pattern = r'\d'
new_pattern = '<digit>'
new_str = re.sub(old_pattern, new_pattern, test_str)
print(new_str)

Python 3.8
Python <digit>.<digit>


### Remove hyperlinks

In [ ]:
test_str = "Visit https://www.amazon.com for more information on this."
test_pattern = r'http\S+'
print(test_str)
new_str = re.sub(test_pattern, " ", test_str)
print(new_str)

Visit https://www.amazon.com for more information on this.
Visit   for more information on this.


In [ ]:
def remove_hyperlinks(text):
    pattern_for_hyperlink = r'http\S+'
    return re.sub(pattern_for_hyperlink, " ", text)

In [ ]:
reviews = [remove_hyperlinks(review) for review in reviews]

In [ ]:
reviews[0:5]

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.']

### Expand contracted words

In [ ]:
def remove_contracted_words(text):
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
test_str = "I can't use this product. I don't recommend this product to anyone."
print(test_str)
print(remove_contracted_words(test_str))

I can't use this product. I don't recommend this product to anyone.
I can not use this product. I do not recommend this product to anyone.


In [ ]:
reviews = [remove_contracted_words(review) for review in reviews]

In [ ]:
reviews[0:5]

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.']

### Remove everything other than letters of alphabet

In [ ]:
def remove_non_letters(text):
    antipattern = r'[^A-Za-z]+'
    return re.sub(antipattern, " ", text)

In [ ]:
reviews = [remove_non_letters(review) for review in reviews]
reviews[0:5]

['I order of them and one of the item is bad quality Is missing backup spring so I have to put a pcs of aluminum to make the battery work ',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy ',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great ']

In [ ]:
test_string = "I recommend      resolution with   GB of RAM     "
test_string_list = test_string.split()
test_string_list

['I', 'recommend', 'resolution', 'with', 'GB', 'of', 'RAM']

In [ ]:
' '.join(test_string_list)

'I recommend resolution with GB of RAM'

In [ ]:
test_string = "       I do not like this product     "
test_string.strip()

'I do not like this product'

In [ ]:
test_string = "I did not like Deliver Package provided by AMAZON"
test_string.lower()

'i did not like deliver package provided by amazon'

In [ ]:
def remove_spaces_and_convert_to_Lowercase(text):
    return ' '.join(text.split()).strip().lower()

In [ ]:
reviews = [remove_spaces_and_convert_to_Lowercase(review) for review in reviews]
reviews[0:5]

['i order of them and one of the item is bad quality is missing backup spring so i have to put a pcs of aluminum to make the battery work',
 'bulk is always the less expensive way to go for products like these',
 'well they are not duracell but for the price i am happy',
 'seem to work as well as name brand batteries at a much better price',
 'these batteries are very long lasting the price is great']

## Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
VOCAB_SIZE = 30000
UNK_TOK = '<UNK>'

In [ ]:
tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token=UNK_TOK)

In [ ]:
tokenizer.fit_on_texts(reviews)

In [ ]:
sequences = tokenizer.texts_to_sequences(reviews)

In [ ]:
print(sequences[1])
print(sequences[5])
print(reviews[1])
print(reviews[5])

[532, 9, 181, 2, 216, 209, 167, 5, 137, 7, 296, 51, 30]
[37, 8, 129, 14, 15, 7, 136, 4, 2, 319, 952, 17, 113, 23, 3, 17, 13, 658, 8, 359, 476, 2, 96, 104, 15, 4, 2, 33, 304, 96, 43, 8, 129, 324, 5, 128, 4, 17, 1974, 47, 2, 362, 4, 17, 24, 375, 48, 58, 141]
bulk is always the less expensive way to go for products like these
bought a lot of batteries for christmas and the amazonbasics cell have been good i have not noticed a difference between the brand name batteries and the amazon basic brand just a lot easier to purchase and have arrive at the house and have on hand will buy again


## Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_LEN = 32 # 32 words in my review

In [ ]:
padded_sequences = np.array(pad_sequences(sequences,
                                          maxlen=MAX_LEN,
                                          padding='post',
                                          truncating='post'))

In [ ]:
print(padded_sequences[0])
print(padded_sequences[1])
print(padded_sequences[3])

[   3  313   14   38    4   42   14    2  252    9  350   92    9 1348
 1410 2172   28    3   17    5  177    8 3575   14 3945    5  240    2
   64   55    0    0]
[532   9 181   2 216 209 167   5 137   7 296  51  30   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[138   5  55  16  53  16 104  96  15  47   8  82  88  25   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


## Create the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 16, input_length=MAX_LEN))
model.add(Bidirectional(SimpleRNN(64, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64)))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 16)            480000    
                                                                 
 bidirectional (Bidirection  (None, 32, 128)           10368     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               24704     
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 3)                 7

## Model training

In [ ]:
model.fit(padded_sequences, labels, epochs=10, validation_split=0.2)

Epoch 1/10
709/709 [==============================] - 35s 42ms/step - loss: 0.3362 - accuracy: 0.8997 - val_loss: 0.2172 - val_accuracy: 0.9307
Epoch 2/10
709/709 [==============================] - 28s 40ms/step - loss: 0.2338 - accuracy: 0.9245 - val_loss: 0.2017 - val_accuracy: 0.9423
Epoch 3/10
709/709 [==============================] - 27s 38ms/step - loss: 0.1696 - accuracy: 0.9437 - val_loss: 0.1682 - val_accuracy: 0.9453
Epoch 4/10
709/709 [==============================] - 28s 40ms/step - loss: 0.1282 - accuracy: 0.9547 - val_loss: 0.1329 - val_accuracy: 0.9599
Epoch 5/10
709/709 [==============================] - 28s 39ms/step - loss: 0.0992 - accuracy: 0.9660 - val_loss: 0.1303 - val_accuracy: 0.9628
Epoch 6/10
709/709 [==============================] - 29s 41ms/step - loss: 0.0795 - accuracy: 0.9726 - val_loss: 0.1272 - val_accuracy: 0.9665
Epoch 7/10
709/709 [==============================] - 29s 41ms/step - loss: 0.0640 - accuracy: 0.9791 - val_loss: 0.0875 - val_accuracy:

## Sentiment prediction using trained model

In [ ]:
positive_sample = 'my husband and uncle they \
loved it and how easy they are to use with fantastic featuresi gave this as a christmas gift to my inla'

negative_sample = 'if ads dont bother you then this may be a decent device purchased this \
for my kid and it was loaded down with so much spam it kept loading it up making \
it slow and laggy plus the carrasoul loadout makes it hard to navigate for kids \
not very kid friendly oh you can pay to remove the ads but it wont remove them all \
buy the samsung better everything'

neutral_sample='it is not that great'

In [ ]:
sample_sequence = tokenizer.texts_to_sequences([neutral_sample])[0]
sample_sequence_padded = pad_sequences([sample_sequence],
                                       maxlen=MAX_LEN,
                                       padding='post',
                                       truncating='post')

In [ ]:
label_encoding={0:'Negative', 1:'Neutral', 2:'Positive'}

In [ ]:
from sklearn.preprocessing import LabelEncoder

predictions = model.predict(sample_sequence_padded, verbose=0)
print(np.round(predictions, 3))
predicted_label = np.argmax(predictions, axis=1)[0]
print("Review:", neutral_sample)
print("Sentiment:", label_encoding[predicted_label])

[[0.003 0.007 0.99 ]]
Review: it is not that great
Sentiment: Positive
